<h2 style="color:green; font-size: 40px;" align="center">Snowflake Initial Setup Notebook<h2>
<h2 style="color:grey" align="center">Setup Roles and Warehouses<h2>


```sql
-- Create the roles
CREATE OR REPLACE ROLE QA_ROLE;
CREATE OR REPLACE ROLE PROD_ROLE;
CREATE OR REPLACE ROLE DBT_ROLE;

-- Create the warehouses with XS size
CREATE OR REPLACE WAREHOUSE QA_WH
  WITH WAREHOUSE_SIZE = 'X-SMALL'
  AUTO_SUSPEND = 60
  AUTO_RESUME = TRUE;

CREATE OR REPLACE WAREHOUSE PROD_WH
  WITH WAREHOUSE_SIZE = 'X-SMALL'
  AUTO_SUSPEND = 60
  AUTO_RESUME = TRUE;

CREATE OR REPLACE WAREHOUSE DBT_WH
  WITH WAREHOUSE_SIZE = 'X-SMALL'
  AUTO_SUSPEND = 60
  AUTO_RESUME = TRUE;

-- Grant access to the QA_ROLE to use the QA_WH
GRANT OPERATE ON WAREHOUSE QA_WH TO ROLE QA_ROLE;
GRANT USAGE ON WAREHOUSE QA_WH TO ROLE QA_ROLE;

-- Grant access to the PROD_ROLE to use the PROD_WH
GRANT OPERATE ON WAREHOUSE PROD_WH TO ROLE PROD_ROLE;
GRANT USAGE ON WAREHOUSE PROD_WH TO ROLE PROD_ROLE;

-- Grant access to the DBT_ROLE to use the DBT_WH
GRANT OPERATE ON WAREHOUSE DBT_WH TO ROLE DBT_ROLE;
GRANT USAGE ON WAREHOUSE DBT_WH TO ROLE DBT_ROLE;

-- Verify the roles and warehouses were created correctly
SHOW ROLES;
SHOW WAREHOUSES;

-- Assign the newly created roles to the account admin for testing purposes
GRANT ROLE QA_ROLE TO USER ACCOUNTADMIN;
GRANT ROLE PROD_ROLE TO USER ACCOUNTADMIN;
GRANT ROLE DBT_ROLE TO USER ACCOUNTADMIN;


<h2 style="color:grey" align="center">Create Raw Database and Source_1 Schema<h2>


```sql
-- Create the "raw" database
CREATE OR REPLACE DATABASE RAW;

-- Create the "source_1" schema within the "raw" database
CREATE OR REPLACE SCHEMA RAW.SOURCE_1;

-- Grant usage and object privileges to the QA_ROLE and DBT_ROLE
GRANT USAGE ON DATABASE RAW TO ROLE QA_ROLE;
GRANT USAGE ON DATABASE RAW TO ROLE DBT_ROLE;

GRANT USAGE ON SCHEMA RAW.SOURCE_1 TO ROLE QA_ROLE;
GRANT USAGE ON SCHEMA RAW.SOURCE_1 TO ROLE DBT_ROLE;

-- Grant object-level privileges for future objects (tables, views) in "source_1" to QA_ROLE and DBT_ROLE
GRANT SELECT, INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA RAW.SOURCE_1 TO ROLE QA_ROLE;
GRANT SELECT, INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA RAW.SOURCE_1 TO ROLE DBT_ROLE;

-- Optionally, grant similar privileges for future views if needed
GRANT SELECT ON FUTURE VIEWS IN SCHEMA RAW.SOURCE_1 TO ROLE QA_ROLE;
GRANT SELECT ON FUTURE VIEWS IN SCHEMA RAW.SOURCE_1 TO ROLE DBT_ROLE;

<h2 style="color:grey" align="center">Create Analytics Database and Schemas<h2>

```sql
-- Create the "analytics" database
CREATE OR REPLACE DATABASE ANALYTICS;

-- Create the "prod" and "qa" schemas within the "analytics" database
CREATE OR REPLACE SCHEMA ANALYTICS.PROD;
CREATE OR REPLACE SCHEMA ANALYTICS.QA;

-- Grant permissions to the QA schema
-- Allow the QA_ROLE and DBT_ROLE to have full access to the QA schema
GRANT USAGE ON SCHEMA ANALYTICS.QA TO ROLE QA_ROLE;
GRANT USAGE ON SCHEMA ANALYTICS.QA TO ROLE DBT_ROLE;

GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA ANALYTICS.QA TO ROLE QA_ROLE;
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA ANALYTICS.QA TO ROLE DBT_ROLE;

-- Grant permissions to the Prod schema
-- Allow the PROD_ROLE and DBT_ROLE to have full access to the Prod schema
GRANT USAGE ON SCHEMA ANALYTICS.PROD TO ROLE PROD_ROLE;
GRANT USAGE ON SCHEMA ANALYTICS.PROD TO ROLE DBT_ROLE;

GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA ANALYTICS.PROD TO ROLE PROD_ROLE;
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA ANALYTICS.PROD TO ROLE DBT_ROLE;

-- Optionally, grant privileges on future objects (tables, views, etc.) in the schemas
GRANT SELECT, INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA ANALYTICS.QA TO ROLE QA_ROLE;
GRANT SELECT, INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA ANALYTICS.QA TO ROLE DBT_ROLE;

GRANT SELECT, INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA ANALYTICS.PROD TO ROLE PROD_ROLE;
GRANT SELECT, INSERT, UPDATE, DELETE ON FUTURE TABLES IN SCHEMA ANALYTICS.PROD TO ROLE DBT_ROLE;


<h2 style="color:grey" align="center">Create Tables in raw.source_1 Schema<h2>

```sql

-- Create the "customer" table in "raw.source_1" schema
CREATE OR REPLACE TABLE RAW.SOURCE_1.CUSTOMER (
    C_CUSTKEY NUMBER,                -- Customer key (numeric)
    C_NAME STRING,                   -- Customer name (string)
    C_ADDRESS STRING,                -- Customer address (string)
    C_NATIONKEY NUMBER,              -- Nation key (numeric)
    C_PHONE STRING,                  -- Phone number (string)
    C_ACCTBAL NUMBER(18, 2),         -- Account balance (numeric with 2 decimal places)
    C_MKTSEGMENT STRING,             -- Market segment (string)
    C_COMMENT STRING                 -- Comment (string)
);


-- Create the LINEITEM table
CREATE OR REPLACE TABLE raw.source_1.lineitem (
    L_ORDERKEY NUMBER,                -- Order key (numeric)
    L_PARTKEY NUMBER,                 -- Part key (numeric)
    L_SUPPKEY NUMBER,                 -- Supplier key (numeric)
    L_LINENUMBER NUMBER,              -- Line number (numeric)
    L_QUANTITY NUMBER(10, 2),         -- Quantity (numeric with decimal)
    L_EXTENDEDPRICE NUMBER(18, 2),    -- Extended price (numeric with decimal)
    L_DISCOUNT NUMBER(4, 2),          -- Discount (numeric with decimal)
    L_TAX NUMBER(4, 2),               -- Tax (numeric with decimal)
    L_RETURNFLAG STRING,              -- Return flag (string)
    L_LINESTATUS STRING,              -- Line status (string)
    L_SHIPDATE DATE,                  -- Ship date (date)
    L_COMMITDATE DATE,                -- Commit date (date)
    L_RECEIPTDATE DATE,               -- Receipt date (date)
    L_SHIPINSTRUCT STRING,            -- Shipping instruction (string)
    L_SHIPMODE STRING,                -- Shipping mode (string)
    L_COMMENT STRING                  -- Comment (string)
);

-- Create the NATION table
CREATE OR REPLACE TABLE raw.source_1.nation (
    N_NATIONKEY NUMBER,               -- Nation key (numeric)
    N_NAME STRING,                    -- Nation name (string)
    N_REGIONKEY NUMBER,               -- Region key (numeric)
    N_COMMENT STRING                  -- Comment (string)
);

-- Create the ORDERS table
CREATE OR REPLACE TABLE raw.source_1.orders (
    O_ORDERKEY NUMBER,                -- Order key (numeric)
    O_CUSTKEY NUMBER,                 -- Customer key (numeric)
    O_ORDERSTATUS STRING,             -- Order status (string)
    O_TOTALPRICE NUMBER(18, 2),       -- Total price (numeric with decimal)
    O_ORDERDATE DATE,                 -- Order date (date)
    O_ORDERPRIORITY STRING,           -- Order priority (string)
    O_CLERK STRING,                   -- Clerk (string)
    O_SHIPPRIORITY NUMBER,            -- Shipping priority (numeric)
    O_COMMENT STRING                  -- Comment (string)
);

-- Create the PART table
CREATE OR REPLACE TABLE raw.source_1.part (
    P_PARTKEY NUMBER,                 -- Part key (numeric)
    P_NAME STRING,                    -- Part name (string)
    P_MFGR STRING,                    -- Manufacturer (string)
    P_BRAND STRING,                   -- Brand (string)
    P_TYPE STRING,                    -- Type (string)
    P_SIZE NUMBER,                    -- Size (numeric)
    P_CONTAINER STRING,               -- Container (string)
    P_RETAILPRICE NUMBER(18, 2),      -- Retail price (numeric with decimal)
    P_COMMENT STRING                  -- Comment (string)
);

-- Create the PARTSUPP table
CREATE OR REPLACE TABLE raw.source_1.partsupp (
    PS_PARTKEY NUMBER,                -- Part key (numeric)
    PS_SUPPKEY NUMBER,                -- Supplier key (numeric)
    PS_AVAILQTY NUMBER,               -- Available quantity (numeric)
    PS_SUPPLYCOST NUMBER(18, 2),      -- Supply cost (numeric with decimal)
    PS_COMMENT STRING                 -- Comment (string)
);

-- Create the REGION table
CREATE OR REPLACE TABLE raw.source_1.region (
    R_REGIONKEY NUMBER,               -- Region key (numeric)
    R_NAME STRING,                    -- Region name (string)
    R_COMMENT STRING                  -- Comment (string)
);

-- Create the SUPPLIER table
CREATE OR REPLACE TABLE raw.source_1.supplier (
    S_SUPPKEY NUMBER,                 -- Supplier key (numeric)
    S_NAME STRING,                    -- Supplier name (string)
    S_ADDRESS STRING,                 -- Supplier address (string)
    S_NATIONKEY NUMBER,               -- Nation key (numeric)
    S_PHONE STRING,                   -- Phone number (string)
    S_ACCTBAL NUMBER(18, 2)           -- Account balance (numeric with decimal)
);


-- Verify the table structures
SHOW TABLES IN SCHEMA raw.source_1;



<h2 style="color:grey" align="center">Inserting data into raw tables<h2>

```sql

-- You can skip this part if not familiar with AWS S3. Proceed and load the files into the table using the Snowflake Interface
-- CSV files are attached

CREATE OR REPLACE FILE FORMAT csv_format
    TYPE = 'CSV'
    FIELD_DELIMITER = ','
    SKIP_HEADER = 1;

-- Insert sample data into the CUSTOMER table
COPY INTO RAW.SOURCE_1.customer
FROM 's3://to_be_defined/file_name.csv'
FILE_FORMAT = csv_format;

-- Load data into the LINEITEM table
COPY INTO raw.source_1.lineitem
FROM 's3://to_be_defined/lineitem.csv'
FILE_FORMAT = csv_format;

-- Load data into the NATION table
COPY INTO raw.source_1.nation
FROM 's3://to_be_defined/nation.csv'
FILE_FORMAT = csv_format;

-- Load data into the ORDERS table
COPY INTO raw.source_1.orders
FROM 's3://to_be_defined/orders.csv'
FILE_FORMAT = csv_format;

-- Load data into the PART table
COPY INTO raw.source_1.part
FROM 's3://to_be_defined/part.csv'
FILE_FORMAT = csv_format;

-- Load data into the PARTSUPP table
COPY INTO raw.source_1.partsupp
FROM 's3://to_be_defined/partsupp.csv'
FILE_FORMAT = csv_format;

-- Load data into the REGION table
COPY INTO raw.source_1.region
FROM 's3://to_be_defined/region.csv'
FILE_FORMAT = csv_format;

-- Load data into the SUPPLIER table
COPY INTO raw.source_1.supplier
FROM 's3://to_be_defined/supplier.csv'
FILE_FORMAT = csv_format;

<h2 style="color:grey" align="center">Validating data<h2>

```sql

-- Select the first 10 rows from the CUSTOMER table
SELECT * FROM raw.source_1.customer LIMIT 10;

-- Select the first 10 rows from the LINEITEM table
SELECT * FROM raw.source_1.lineitem LIMIT 10;

-- Select the first 10 rows from the NATION table
SELECT * FROM raw.source_1.nation LIMIT 10;

-- Select the first 10 rows from the ORDERS table
SELECT * FROM raw.source_1.orders LIMIT 10;

-- Select the first 10 rows from the PART table
SELECT * FROM raw.source_1.part LIMIT 10;

-- Select the first 10 rows from the PARTSUPP table
SELECT * FROM raw.source_1.partsupp LIMIT 10;

-- Select the first 10 rows from the REGION table
SELECT * FROM raw.source_1.region LIMIT 10;

-- Select the first 10 rows from the SUPPLIER table
SELECT * FROM raw.source_1.supplier LIMIT 10;


```sql